In [1]:
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics import mean_squared_error
from keras.initializers import RandomNormal
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.models import Model,Sequential
from keras.layers import *
from keras import backend as K
from keras.models import model_from_json
from matplotlib import cm as CM
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import scipy.io as io
from PIL import Image
import PIL
import h5py
import os
import glob
import cv2
import random
import math
import sys
import numpy as np

In [2]:
K.clear_session()
root = os.path.join(os.getcwd(),'ShanghaiTech')
print(root)

C:\Users\mohit\MINI PROJ\ShanghaiTech


In [3]:
part_A_train = os.path.join(root,'part_A/train_data','images')
part_A_test = os.path.join(root,'part_A/test_data','images')
part_B_train = os.path.join(root,'part_B/train_data','images')
part_B_test = os.path.join(root,'part_B/test_data','images')
temp = 'test_images'
path_sets = [part_A_train]

In [4]:
img_paths = []

for path in path_sets:
    
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        
        img_paths.append(str(img_path))
        
print("Total images : ",len(img_paths))

Total images :  300


In [5]:
def create_img(path):
    #Function to load,normalize and return image 
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225

    #print(im.shape)
    #im = np.expand_dims(im,axis  = 0)
    return im

def get_input(path):
    path = path[0] 
    img = create_img(path)
    return(img)
    
    
    
def get_output(path):
    #import target
    #resize target
    
    gt_file = h5py.File(path,'r')
    
    target = np.asarray(gt_file['density'])
    
    img = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64
    
    img = np.expand_dims(img,axis  = 2)
    
    #print(img.shape)
    
    return img
    
    
    
def preprocess_input(image,target):
    #crop image
    #crop target
    #resize target
    crop_size = (int(image.shape[0]/2),int(image.shape[1]/2))
    
    
    if random.randint(0,9)<= -1:            
            dx = int(random.randint(0,1)*image.shape[0]*1./2)
            dy = int(random.randint(0,1)*image.shape[1]*1./2)
    else:
            dx = int(random.random()*image.shape[0]*1./2)
            dy = int(random.random()*image.shape[1]*1./2)

    #print(crop_size , dx , dy)
    img = image[dx : crop_size[0]+dx , dy:crop_size[1]+dy]
    
    target_aug = target[dx:crop_size[0]+dx,dy:crop_size[1]+dy]
    #print(img.shape)

    return(img,target_aug)

In [6]:
#Image data generator 
def image_generator(files, batch_size = 64):
    
    while True:
        
        input_path = np.random.choice(a = files, size = batch_size)
        
        batch_input = []
        batch_output = [] 
          
        #for input_path in batch_paths:
        
        inputt = get_input(input_path )
        output = get_output(input_path[0].replace('.jpg','.h5').replace('images','ground_truth') )
            
       
        batch_input += [inputt]
        batch_output += [output]
    

        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )

In [7]:
def save_mod(model , str1 , str2):
    model.save_weights(str1)
    
    model_json = model.to_json()
    
    with open(str2, "w") as json_file:
        json_file.write(model_json)

In [8]:
def init_weights_vgg(model):
    #vgg =  VGG16(weights='imagenet', include_top=False)
    
    json_file = open('VGG_16.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("VGG_16.h5")
    
    vgg = loaded_model
    
    vgg_weights=[]                         
    for layer in vgg.layers:
        if('conv' in layer.name):
            vgg_weights.append(layer.get_weights())
    
    
    offset=0
    i=0
    while(i<10):
        if('conv' in model.layers[i+offset].name):
            model.layers[i+offset].set_weights(vgg_weights[i])
            i=i+1
            #print('h')
            
        else:
            offset=offset+1

    return (model)

In [9]:
def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [12]:
# Neural network model : VGG + Conv
def CrowdNet():  
            #Variable Input Size
            rows = None
            cols = None
            
            #Batch Normalisation option
            
            batch_norm = 0
            kernel = (3, 3)
            init = RandomNormal(stddev=0.01)
            model = Sequential() 
            
            #custom VGG:
            
            if(batch_norm):
                model.add(Conv2D(64, kernel_size = kernel, input_shape = (rows,cols,3),activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                
            else:
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same',input_shape = (rows, cols, 3), kernel_initializer = init))
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                
                

                
            #Conv2D
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(1, (1, 1), activation='relu', dilation_rate = 1, kernel_initializer = init, padding = 'same'))
        
            #sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
            opt = Adam(learning_rate=0.01)
            model.compile(optimizer=opt, loss=euclidean_distance_loss, metrics=['mse'])
            
            
            model = init_weights_vgg(model)
            
            return model

In [13]:
model = CrowdNet()

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, None, None, 128)   147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, None, None, 128)   0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, None, None, 256)  

In [15]:
train_gen = image_generator(img_paths,1)

In [18]:
opt = Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=euclidean_distance_loss, metrics=['mse'])

In [19]:
model.fit_generator(train_gen,epochs=2,steps_per_epoch= 700 , verbose=1)

C:\Users\mohit\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2


226/700 [========>.....................] - ETA: 17:43:12 - loss: 0.0099 - mse: 0.004 - ETA: 2:59:10 - loss: nan - mse: nan      - ETA: 3:06:50 - loss: nan - mse: n - ETA: 2:55:25 - loss: nan - mse: n - ETA: 2:36:27 - loss: nan - mse: n - ETA: 2:16:32 - loss: nan - mse: n - ETA: 2:31:50 - loss: nan - mse: n - ETA: 2:19:55 - loss: nan - mse: n - ETA: 2:19:05 - loss: nan - mse: n - ETA: 2:11:25 - loss: nan - mse: n - ETA: 2:06:10 - loss: nan - mse: n - ETA: 2:11:19 - loss: nan - mse: n - ETA: 2:12:24 - loss: nan - mse: n - ETA: 2:14:51 - loss: nan - mse: n - ETA: 2:21:05 - loss: nan - mse: n - ETA: 2:15:46 - loss: nan - mse: n - ETA: 2:11:00 - loss: nan - mse: n - ETA: 2:12:13 - loss: nan - mse: n - ETA: 2:16:20 - loss: nan - mse: n - ETA: 2:15:46 - loss: nan - mse: n - ETA: 2:19:23 - loss: nan - mse: n - ETA: 2:21:24 - loss: nan - mse: n - ETA: 2:24:40 - loss: nan - mse: n - ETA: 2:24:55 - loss: nan - mse: n - ETA: 2:27:31 - loss: nan - mse: n - ETA: 2:23:00 - loss: nan - mse: n - ETA: 2

459/700 [==================>...........] - ETA: 1:31:55 - loss: nan - mse: n - ETA: 1:31:26 - loss: nan - mse: n - ETA: 1:31:11 - loss: nan - mse: n - ETA: 1:30:58 - loss: nan - mse: n - ETA: 1:30:45 - loss: nan - mse: n - ETA: 1:30:26 - loss: nan - mse: n - ETA: 1:30:11 - loss: nan - mse: n - ETA: 1:29:59 - loss: nan - mse: n - ETA: 1:29:46 - loss: nan - mse: n - ETA: 1:29:32 - loss: nan - mse: n - ETA: 1:29:06 - loss: nan - mse: n - ETA: 1:28:53 - loss: nan - mse: n - ETA: 1:28:34 - loss: nan - mse: n - ETA: 1:28:22 - loss: nan - mse: n - ETA: 1:28:12 - loss: nan - mse: n - ETA: 1:27:49 - loss: nan - mse: n - ETA: 1:27:38 - loss: nan - mse: n - ETA: 1:27:13 - loss: nan - mse: n - ETA: 1:26:47 - loss: nan - mse: n - ETA: 1:26:35 - loss: nan - mse: n - ETA: 1:26:14 - loss: nan - mse: n - ETA: 1:26:00 - loss: nan - mse: n - ETA: 1:25:47 - loss: nan - mse: n - ETA: 1:25:35 - loss: nan - mse: n - ETA: 1:25:10 - loss: nan - mse: n - ETA: 1:25:08 - loss: nan - mse: n - ETA: 1:24:41 - loss: 

699/700 [============================>.] - ETA: 40:41 - loss: nan - mse: n - ETA: 40:32 - loss: nan - mse: n - ETA: 40:21 - loss: nan - mse: n - ETA: 40:11 - loss: nan - mse: n - ETA: 40:02 - loss: nan - mse: n - ETA: 39:50 - loss: nan - mse: n - ETA: 39:40 - loss: nan - mse: n - ETA: 39:29 - loss: nan - mse: n - ETA: 39:19 - loss: nan - mse: n - ETA: 39:12 - loss: nan - mse: n - ETA: 39:03 - loss: nan - mse: n - ETA: 38:54 - loss: nan - mse: n - ETA: 38:45 - loss: nan - mse: n - ETA: 38:36 - loss: nan - mse: n - ETA: 38:26 - loss: nan - mse: n - ETA: 38:16 - loss: nan - mse: n - ETA: 38:04 - loss: nan - mse: n - ETA: 37:55 - loss: nan - mse: n - ETA: 37:43 - loss: nan - mse: n - ETA: 37:34 - loss: nan - mse: n - ETA: 37:20 - loss: nan - mse: n - ETA: 37:09 - loss: nan - mse: n - ETA: 36:55 - loss: nan - mse: n - ETA: 36:42 - loss: nan - mse: n - ETA: 36:32 - loss: nan - mse: n - ETA: 36:20 - loss: nan - mse: n - ETA: 36:10 - loss: nan - mse: n - ETA: 35:58 - loss: nan - mse: n - ETA: 

700/700 [==============================] - ETA: 0s - loss: nan - mse: nan - 7784s 11s/step - loss: nan - mse: nan
Epoch 2/2


227/700 [========>.....................] - ETA: 3:38:58 - loss: nan - mse: n - ETA: 4:03:37 - loss: nan - mse: n - ETA: 3:50:02 - loss: nan - mse: n - ETA: 2:57:57 - loss: nan - mse: n - ETA: 3:07:29 - loss: nan - mse: n - ETA: 3:06:34 - loss: nan - mse: n - ETA: 3:14:56 - loss: nan - mse: n - ETA: 2:57:51 - loss: nan - mse: n - ETA: 3:02:24 - loss: nan - mse: n - ETA: 2:56:32 - loss: nan - mse: n - ETA: 2:57:57 - loss: nan - mse: n - ETA: 2:54:10 - loss: nan - mse: n - ETA: 2:42:44 - loss: nan - mse: n - ETA: 2:46:26 - loss: nan - mse: n - ETA: 2:40:46 - loss: nan - mse: n - ETA: 2:45:57 - loss: nan - mse: n - ETA: 2:48:08 - loss: nan - mse: n - ETA: 2:48:10 - loss: nan - mse: n - ETA: 2:52:08 - loss: nan - mse: n - ETA: 2:46:42 - loss: nan - mse: n - ETA: 2:46:30 - loss: nan - mse: n - ETA: 2:46:24 - loss: nan - mse: n - ETA: 2:42:26 - loss: nan - mse: n - ETA: 2:36:41 - loss: nan - mse: n - ETA: 2:37:22 - loss: nan - mse: n - ETA: 2:38:28 - loss: nan - mse: n - ETA: 2:34:49 - loss: 

455/700 [==================>...........] - ETA: 2:17:14 - loss: nan - mse: n - ETA: 2:16:42 - loss: nan - mse: n - ETA: 2:16:14 - loss: nan - mse: n - ETA: 2:15:34 - loss: nan - mse: n - ETA: 2:14:58 - loss: nan - mse: n - ETA: 2:14:15 - loss: nan - mse: n - ETA: 2:13:30 - loss: nan - mse: n - ETA: 2:13:00 - loss: nan - mse: n - ETA: 2:12:31 - loss: nan - mse: n - ETA: 2:12:01 - loss: nan - mse: n - ETA: 2:11:32 - loss: nan - mse: n - ETA: 2:11:04 - loss: nan - mse: n - ETA: 2:10:20 - loss: nan - mse: n - ETA: 2:09:54 - loss: nan - mse: n - ETA: 2:09:16 - loss: nan - mse: n - ETA: 2:08:45 - loss: nan - mse: n - ETA: 2:08:17 - loss: nan - mse: n - ETA: 2:07:48 - loss: nan - mse: n - ETA: 2:07:19 - loss: nan - mse: n - ETA: 2:06:51 - loss: nan - mse: n - ETA: 2:06:19 - loss: nan - mse: n - ETA: 2:05:38 - loss: nan - mse: n - ETA: 2:05:14 - loss: nan - mse: n - ETA: 2:04:49 - loss: nan - mse: n - ETA: 2:04:21 - loss: nan - mse: n - ETA: 2:03:54 - loss: nan - mse: n - ETA: 2:03:17 - loss: 

695/700 [============================>.] - ETA: 53:07 - loss: nan - mse: n - ETA: 52:54 - loss: nan - mse: n - ETA: 52:35 - loss: nan - mse: n - ETA: 52:19 - loss: nan - mse: n - ETA: 52:06 - loss: nan - mse: n - ETA: 51:49 - loss: nan - mse: n - ETA: 51:33 - loss: nan - mse: n - ETA: 51:18 - loss: nan - mse: n - ETA: 51:02 - loss: nan - mse: n - ETA: 50:46 - loss: nan - mse: n - ETA: 50:32 - loss: nan - mse: n - ETA: 50:17 - loss: nan - mse: n - ETA: 50:03 - loss: nan - mse: n - ETA: 49:48 - loss: nan - mse: n - ETA: 49:34 - loss: nan - mse: n - ETA: 49:18 - loss: nan - mse: n - ETA: 49:05 - loss: nan - mse: n - ETA: 48:52 - loss: nan - mse: n - ETA: 48:38 - loss: nan - mse: n - ETA: 48:24 - loss: nan - mse: n - ETA: 48:10 - loss: nan - mse: n - ETA: 47:56 - loss: nan - mse: n - ETA: 47:42 - loss: nan - mse: n - ETA: 47:25 - loss: nan - mse: n - ETA: 47:07 - loss: nan - mse: n - ETA: 46:53 - loss: nan - mse: n - ETA: 46:40 - loss: nan - mse: n - ETA: 46:27 - loss: nan - mse: n - ETA: 

700/700 [==============================] - ETA: 46s - loss: nan - mse: n - ETA: 35s - loss: nan - mse: n - ETA: 23s - loss: nan - mse: n - ETA: 11s - loss: nan - mse: n - ETA: 0s - loss: nan - mse: nan - 8171s 12s/step - loss: nan - mse: nan


In [20]:
  save_mod(model,"model_A_weights2.h5","Model.json")